In [1]:
import pandas as pd
import numpy as np
import os, glob, sys
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

# device number 설정
device = torch.device('cuda:{}'.format(0) if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU 

import warnings
warnings.filterwarnings(action='ignore') 

## 500 mtorr 

In [2]:
df = pd.read_csv('/home/data/psk/leakage/ID_9.csv')
df = df.dropna(axis = 0)
df = df[df.Torr == 500]

df.TimeStamp = pd.to_datetime(df.TimeStamp)
df = df.sort_values(by = ['TimeStamp'])
col_list = ['TimeStamp','PM1.Gas1_Monitor', 'PM1.Gas2_Monitor', 'PM1.APC_Pressure', 'PM1.APC_Position', 
    'PM1.SourcePwr1_Read', 'PM1.SourcePwr2_Read', 'PM1.Temp1', 'PM1.Temp2',  
    'ApcPosition', 'ApcPositionScaled','WallTemp',
    'Port_Num', 'Process_Num', 'Wafer_Status', 'Folder_Name', 'File_Name', 'Torr','Leak']
df_ = df.loc[:, col_list]

wafer_unit = list(df_.groupby(['Port_Num', 'Process_Num', 'Wafer_Status', 'Folder_Name', 'File_Name'])) # KEY 값
wafer_list = list(filter(lambda x: 102 > len(x[1]) > 90, wafer_unit))    

print(pd.Series([wafer[1].Leak.unique()[0] for wafer in wafer_list]).value_counts())
print(set([len(wafer[1]) for wafer in wafer_list]))
print(pd.Series([len(wafer[1]) for wafer in wafer_list]).value_counts())

0    5214
1    1141
dtype: int64
{99, 101}
101    6354
99        1
dtype: int64


## Split train, test, valid

In [3]:
from sklearn.model_selection import train_test_split

X_train_, X_test, y_train, y_test = train_test_split(wafer_list, range(len(wafer_list)), test_size = 0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_, range(len(X_train_)), test_size = 0.1, random_state=42)

split_data = [X_train, X_valid, X_test]

for data in split_data:
    print(pd.Series([wafer[1].Leak.unique()[0] for wafer in data]).value_counts())

0    4213
1     934
dtype: int64
0    478
1     94
dtype: int64
0    523
1    113
dtype: int64


## Dataset with mean std

## Traing 기반 MinMaxScaler 적용
### 각 데이터 셋에 적용

In [4]:
from sklearn.preprocessing import MinMaxScaler

mean_std = []
datasets = [X_train, X_valid, X_test]

for idx, dataset in tqdm(enumerate(datasets)):
    
    sam = []
    
    if idx == 0:
        
        m_scaler = MinMaxScaler()
        # 해당 dataset 의 mean 과 std 구하기
        mean = np.array([df.iloc[:, 1:-7].mean().values for info, df in dataset])
        scale_mean = m_scaler.fit_transform(mean)
        sam.append(scale_mean)

        s_scaler = MinMaxScaler() 
        std = np.array([df.iloc[:, 1:-7].std().values for info, df in dataset])
        scale_std = s_scaler.fit_transform(std)
        sam.append(scale_std)
        mean_std.append(sam)
        
    else:

        mean = np.array([df.iloc[:, 1:-7].mean().values for info, df in dataset])
        scale_mean = m_scaler.transform(mean)
        sam.append(scale_mean)
        
        std = np.array([df.iloc[:, 1:-7].std().values for info, df in dataset])
        scale_std = s_scaler.transform(std)
        sam.append(scale_std)

        mean_std.append(sam)

3it [00:07,  2.62s/it]


In [5]:
from sklearn.preprocessing import StandardScaler

class datasetAE(Dataset):
    def __init__(self, wafer_unit, window_size, max_len, mean_arr, std_arr):
        
        super(datasetAE, self).__init__()
        self.wafer_unit = wafer_unit
        self.window_size = window_size
        self.max_len = max_len
        self.mean_arr = mean_arr
        self.std_arr = std_arr
    
    def __getitem__(self, idx):
        
        df = self.wafer_unit[idx][1]
        info = self.wafer_unit[idx][0]
        torr = df.Torr.unique()[0]
        y = df.Leak.unique()[0]
        
        df_ = df.iloc[:, 1:-7].reset_index(drop=True)
        
        mean_ = self.mean_arr[idx]
        std_ = self.std_arr[idx]
        
        # zero padding
        if len(df_) < self.max_len:
            
            add_num = abs(len(df_) - self.max_len)
            for num in range(add_num):
                df_.loc[len(df_)] = 0
        
        scaler = StandardScaler()                              
        df__ = pd.DataFrame(scaler.fit_transform(df_), columns = df_.columns).reset_index(drop=-True)
                
        x = np.array(df__).T
        
        # sliding window 적용
        windows_x = np.lib.stride_tricks.sliding_window_view(x, self.window_size, 1)
        
        sensor_list = []
        for idx, sensor in enumerate(windows_x):
            add_mean = np.full(self.window_size, mean_[idx])
            add_std = np.full(self.window_size, std_[idx])
            sensor = np.vstack([sensor, add_mean, add_std])
            sensor_list.append(sensor)
        
        return np.array(sensor_list), np.array(y), torr
    
    def __len__(self):
        return len(self.wafer_unit)

In [6]:
# mean_std[0] : train_X, [1] : Valid_X, [2] : Test_X 
## mean_std[0][1] : train_X 의 센서별 scaled mean 값
## mean_std[0][2] : train_X 의 센서별 scaled std 값

max_len = max(set([len(wafer[1]) for wafer in wafer_list]))
train_dataset = datasetAE(X_train, 70, max_len, mean_std[0][0], mean_std[0][1])
valid_dataset = datasetAE(X_valid, 70, max_len, mean_std[1][0], mean_std[1][1])
test_dataset = datasetAE(X_test, 70, max_len, mean_std[2][0], mean_std[2][1])

train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=16, drop_last=True)

## Model

In [7]:
class MTS_CNN(torch.nn.Module):
    
    # 3 class 모델
    def __init__(self, sensor_num):
        super(MTS_CNN, self).__init__()
        
        extract_list = []
        diagnosis_list = []
        for num in range(sensor_num):
            
            extract_layer = torch.nn.Sequential(
                    torch.nn.Conv1d(in_channels = 32, out_channels = 16, 
                                    kernel_size = 5),
                    torch.nn.ReLU(),
                    torch.nn.AvgPool1d(kernel_size = 3, stride = 3),
                    torch.nn.Conv1d(in_channels = 16, out_channels = 64, 
                                    kernel_size = 5),
                    torch.nn.ReLU(),
                    torch.nn.AvgPool1d(kernel_size = 3, stride = 3),
            )

            
            diagnosis_layer = torch.nn.Sequential(
                     torch.nn.Linear(386, 256),
                     torch.nn.ReLU(),
                     torch.nn.Linear(256, 1),
                     torch.nn.ReLU(),
            )
            
            extract_list.append(extract_layer)
            diagnosis_list.append(diagnosis_layer)
            
        
        self.extract_layer = nn.ModuleList(extract_list)
        self.diagnosis_layer = nn.ModuleList(diagnosis_list)
        
        self.detection_layer = torch.nn.Sequential(
                                             torch.nn.Linear(sensor_num, 32),
                                             torch.nn.ReLU(),
                                             torch.nn.Linear(32, 16),
                                             torch.nn.Dropout(0.5),
                                             torch.nn.ReLU(),
                                             torch.nn.Linear(16, 2)
                                    )
    
    def forward(self, x):
        b, s, q, w = x.shape
        # s, b, q, w
        x_ = x.transpose(1,0)
        
        stack_sensor = []
        # 센서 별로 convolution 적용
        for idx, x__ in enumerate(x_):
            
            # seq 만 뺴오기
            # 32 
            seq = x__[:, :-2, :]
            
            # mean, std, 분리
            mean = x__[:, -2, 0].view(-1,1)
            std = x__[:, -1, 0].view(-1,1)
            
            # Feature Extraction Layer
            feature = self.extract_layer[idx](seq)
            flatten = feature.view(b, -1)
            
            # 1024 (shape) + 2 (mean, std) = 1026
            flatten_ = torch.cat((flatten, mean, std), 1)
            
            # Fault Diagnosis Layer
            diagnosis = self.diagnosis_layer[idx](flatten_)
            stack_sensor.append(diagnosis)
        
        # Stack By Sensor
        fin_stack = torch.stack(stack_sensor).transpose(0,1).flatten(start_dim = -2)
        
        # Fault Detection Layer
        output = self.detection_layer(fin_stack)
        
        return output, fin_stack

In [8]:
test_tensor = torch.rand(64, 11, 34, 70) # b, s, q, w
model = MTS_CNN(11)
model(test_tensor)

(tensor([[-0.3107,  0.1192],
         [-0.2952,  0.0900],
         [-0.3613,  0.0688],
         [-0.3578,  0.0624],
         [-0.3150,  0.1078],
         [-0.2541,  0.1270],
         [-0.3024,  0.0884],
         [-0.3007,  0.1004],
         [-0.2972,  0.1014],
         [-0.2318,  0.1342],
         [-0.2542,  0.1258],
         [-0.3012,  0.0867],
         [-0.3158,  0.1009],
         [-0.3184,  0.0990],
         [-0.3328,  0.1098],
         [-0.2322,  0.1350],
         [-0.3285,  0.1095],
         [-0.2181,  0.1213],
         [-0.3180,  0.1016],
         [-0.2332,  0.1201],
         [-0.2231,  0.1233],
         [-0.2343,  0.1273],
         [-0.2464,  0.1336],
         [-0.2328,  0.1192],
         [-0.2852,  0.0957],
         [-0.2549,  0.1253],
         [-0.2585,  0.1259],
         [-0.1746,  0.1522],
         [-0.1989,  0.1522],
         [-0.2528,  0.1169],
         [-0.2687,  0.1295],
         [-0.2342,  0.1339],
         [-0.1941,  0.1498],
         [-0.2994,  0.1075],
         [-0.1

## Train

In [9]:
from datetime import datetime

def save_model_by_date(model, optimizer, model_name, save_path, epoch, n_epochs, avg_valid_losses):
    
    global new 
    
    now = datetime.now().strftime('%Y_%m_%d') 
    date_save_path = save_path + '/' + now + '/'
    
    if not os.path.exists(date_save_path):
        os.makedirs(date_save_path)
        new = 1
        
    if epoch == 1:
        new = 0
        if len(glob.glob(date_save_path + '*.pt')) > 0:
            for ex_model in glob.glob(date_save_path + '*.pt'):
                os.remove(ex_model)
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))
        else:
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))
    
    elif new == 1:
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))
        new = 0
    
    elif n_epochs == epoch:
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_last_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))
    
    else:
        if avg_valid_losses[-1] < np.min(avg_valid_losses[:-1]):
            print('updated model saved!')
            ex_model = glob.glob(date_save_path + '*.pt')[0]
            os.remove(ex_model)
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(date_save_path, '{}_epoch_{}_valid_loss_{}.pt'.format(model_name, str(epoch), str(avg_valid_losses[-1]))))

In [10]:
from sklearn.metrics import confusion_matrix

def train_model(model, model_name, save_path, n_epochs, device, train_loader, valid_loader, optimizer, criterion):
    
    model.to(device)
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    avg_valid_losses = []
    
    print('start-training')
    for epoch in range(1, n_epochs + 1):
        
        # to track the training loss as the model trains
        train_losses = []
        valid_losses = []
        
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        for batch, (data, label, torr) in tqdm(enumerate(train_loader, 1)):
            data = data.float().to(device)
            label = label.to(device)
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output, fin_stack = model(data)
            # calculate the loss
            loss = criterion(output, label)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
#             print(list(model.layer_dict['0'][0].parameters())[0][0][0])
#             print(list(model.layer_dict['0'][1].parameters())[0][0])
            
            # record training loss
            train_losses.append(loss.item())
            
        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        
        print('epochs : {} / avg_train_loss : {} '.format(epoch, train_loss))
        
        avg_train_losses.append(train_loss)
        
        model.eval()
        with torch.no_grad(): 
            for batch, (data, label, torr) in tqdm(enumerate(valid_loader, 1)):
                data = data.float().to(device)
                label = label.to(device)
                output, fin_stack = model(data)
                loss = criterion(output, label)
                valid_losses.append(loss.item())

            valid_loss = np.average(valid_losses)

            print('epochs : {} / avg_valid_loss : {} '.format(epoch, valid_loss))

            avg_valid_losses.append(valid_loss)
            
        if epoch == 1:
            pass
        else:
            if avg_valid_losses[-1] < np.min(avg_valid_losses[:-1]):
                print('updated model saved!')
                model_info = [epoch, model.state_dict()]
                
        save_model_by_date(model, optimizer, model_name, save_path, epoch, n_epochs, avg_valid_losses)

    return  model, avg_train_losses, avg_valid_losses, model_info

In [ ]:
model = MTS_CNN(11).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

n_epochs = 200
# device = 0
model_name = 'CLF'
save_path = './Leakage_model/MTS-CNN_MS' # 저장 장소 + 오늘 날짜 저장

model, avg_train_losses, avg_valid_losses, model_info = train_model(model, model_name, save_path, n_epochs, 
                                                                    device, train_loader, valid_loader, optimizer, criterion)

## Model Test

In [ ]:
model = MTS_CNN(11).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
best_model_path = './Leakage_model/MTS-CNN_MS/2021_11_16' # 모델 저장된 경로 입력
model_pt = glob.glob(best_model_path + '/*')[-1]
print(model_pt)

checkpoint = torch.load(model_pt)
model.load_state_dict((checkpoint['model_state_dict']))
print('<All keys matched successfully>')

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def test_model(model, test_dl, device):

    target_list = []
    predict_list = []
    dia_list = []
    
    model.eval()
    with torch.no_grad():
        for num, data in enumerate(tqdm(test_dl)):

            data, labels, torr = data

            data = data.float().to(device)
            labels = labels.to(device)

            outputs, fin_stack= model(data)
            predicted = torch.argmax(outputs.data, axis = 0 )
            
            target_list.append(labels.cpu().numpy())
            predict_list.append(predicted.cpu().numpy())

            dia_list.append(fin_stack.cpu())

        fin_target = np.concatenate(target_list, axis = 0)
        fin_pred = np.array(predict_list)
        fin_dia = torch.stack(dia_list)
                
        print(confusion_matrix(fin_target, fin_pred))
        print(classification_report(fin_target, fin_pred))
        
    return fin_target, fin_pred, np.array(fin_dia.view(-1, 11))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def test_model(model, test_dl, device):

    target_list = []
    predict_list = []
    dia_list = []
    
    model.eval()
    with torch.no_grad():
        for num, data in enumerate(tqdm(test_dl)):

            data, labels, torr = data

            data = data.float().to(device)
            labels = labels.to(device)
            
            outputs, fin_stack = model(data)
            predicted = torch.argmax(outputs.data, axis = 1)
            
            print(outputs, predicted)
            sys.exit()

            target_list.append(labels.cpu().numpy())
            predict_list.append(predicted.cpu().numpy())
                
        fin_target = np.concatenate(target_list, axis = 0)
        fin_pred = np.concatenate(predict_list, axis = 0)
        
        print(confusion_matrix(fin_target, fin_pred))
        print(classification_report(fin_target, fin_pred))
        
    return fin_target, fin_pred

In [ ]:
fin_target, fin_pred, fin_dia = test_model(model, test_loader, device)

## Get TSNE graph of fault diagnosis outputs

In [ ]:
from sklearn.manifold import TSNE

embedded = TSNE(n_components=2, learning_rate= 200).fit_transform(fin_dia)
fin_embed = np.concatenate([embedded, fin_target.reshape(-1, 1)] ,axis = 1)
df_embed = pd.DataFrame(fin_embed, columns = ['x', 'y', 'label'])

plt.figure(figsize = [15, 10])
sns.scatterplot(data = df_embed, hue = 'label', x = 'x', y = 'y')
plt.legend()
plt.show()

## Get sensor importance

In [ ]:
for name, param in model.named_parameters():
    if name == 'detection_layer.0.weight':
        weight = param

weights = weight.detach().cpu().numpy()
weights_ = weights.reshape(11, -1)

col_list = ['PM1.Gas1_Monitor', 'PM1.Gas2_Monitor', 'PM1.APC_Pressure', 'PM1.APC_Position', 
    'PM1.SourcePwr1_Read', 'PM1.SourcePwr2_Read', 'PM1.Temp1', 'PM1.Temp2',  
    'ApcPosition', 'ApcPositionScaled','WallTemp']

plt.figure(figsize = [15, 10])
box_list = []
for w in weights_:
    box_list.append(w)
    
plt.boxplot(box_list)
plt.xticks(range(1, 12), col_list,  rotation=20)
plt.show()